#### ⚙️ Importações

In [1]:
# !pip install pycaret
# !pip install Unidecode

In [2]:
# from google.colab import drive

# drive.mount('/content/drive')
# df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/censo_cultural.xlsx')

In [57]:
from diskcache import Cache
from appdirs import user_cache_dir

cache = Cache(user_cache_dir("cache"))
DATASET_PATH = cache.get("DATASET_PATH")
GENERATED_DATASET_PATH = cache.get("GENERATED_DATASET_PATH")

In [58]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [59]:
# df = pd.read_excel('/mnt/g/Meu Drive/Colab Notebooks/censo_cultural.xlsx')
df = pd.read_excel(f'{DATASET_PATH}/censo_cultural.xlsx')

#### 🔌 Functions

##### Renomear colunas

In [6]:
def rename_columns() -> None:
    df.rename(
        columns={
            "2) Qual o seu curso?": "curso",
            "3) Qual o Ano que está cursando? ": "serie",
            "5) Você é um aluno(a)...  ": "tipo_residente",
            "6) Quais dessas ATIVIDADES CULTURAIS você participa ou já participou?": "atv_culturais",
            "7) Quais dessas HABILIDADES MUSICAIS você possui?": "hab_musicais",
            "8) Quais desses ESPORTES você pratica?": "esportes",
            "9) Quais desses JOGOS você pratica?": "jogos",
            "10) Quais dessas LUTAS você pratica?": "lutas",
            "11) Em relação às 5 perguntas anteriores, quais dessas atividades você se considera bom e capaz de ensinar aos seus colegas?": "melhores_atividades",  # remover
            "12) Quais dessas ATIVIDADES CULTURAIS você gostaria de participar?": "atv_culturais_a_fazer",
            "13) Quais dessas HABILIDADES MUSICAIS você gostaria de possuir?": "hab_musicais_desejadas",
            "14) Quais desses ESPORTES você gostaria de praticar?": "esportes_desejados",
            "15) Quais desses JOGOS você gostaria de praticar?": "jogos_desejados",
            "16) Quais dessas LUTAS você gostaria de praticar?": "lutas_desejadas",
            "17) Em quais HORÁRIOS você teria disponibilidades para exercer Atividades Culturais?": "horarios_disponiveis",  # remover
            "18) Em quais LOCAIS você teria disponibilidades para exercer Atividades Culturais ": "locais_disponiveis",  # remover
            "19) Em relação às ESTRUTURAS EXISTENTES, o que você gostaria que melhorasse em nossa instituição?": "melhoria_estrutura",
            "20) Em relação às NOVAS ESTRUTURAS, o que você desejaria que houvesse em nossa instituição?": "novas_estruturas",
            "21) Em relação à ALIMENTAÇÃO, o que você desejaria que fosse melhorado em nossa instituição?": "melhoria_alimentacao",
            "22) Em relação ao TRANSPORTE, o que você desejaria que fosse melhorado?": "transporte",
            "23) Quais desse PROJETOS ESPECIAIS, você teria interesse em participar?": "projetos_a_participar",
        },
        inplace=True,
    )

##### Tratamento de strings

In [7]:
def encoder(series_names: list, init_label=0) -> None:
   for serie_name in series_names:
      enc = LabelEncoder()
      df[serie_name] = enc.fit_transform(df[serie_name]) + init_label

In [8]:
import unidecode as uni

def remove_accent(name):
    return uni.unidecode(str(name))

##### Dummies

In [9]:
def insert_dummies(df: pd.DataFrame, dummies: pd.DataFrame, insert_position: int) -> None:
    df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
    df.drop('aux', axis=1, inplace=True)

#### 📑 Pré-processamento dos dados

In [10]:
rename_columns()

In [11]:
df.drop(
    columns=[
        "Carimbo de data/hora",
        "1) Nome Completo: ",
        "4) Qual a sua Turma? ",
        "melhores_atividades",
        "horarios_disponiveis",
        "locais_disponiveis",
    ],
    inplace=True,
)

In [12]:
for column in df.columns:
    df[column] = df[column].str.lower()
    df[column] = df[column].apply(remove_accent)

In [13]:
encoder(['serie'], 1)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   curso                   611 non-null    object
 1   serie                   611 non-null    int64 
 2   tipo_residente          611 non-null    object
 3   atv_culturais           611 non-null    object
 4   hab_musicais            611 non-null    object
 5   esportes                611 non-null    object
 6   jogos                   611 non-null    object
 7   lutas                   611 non-null    object
 8   atv_culturais_a_fazer   611 non-null    object
 9   hab_musicais_desejadas  611 non-null    object
 10  esportes_desejados      611 non-null    object
 11  jogos_desejados         611 non-null    object
 12  lutas_desejadas         611 non-null    object
 13  melhoria_estrutura      611 non-null    object
 14  novas_estruturas        611 non-null    object
 15  melhor

#### 📑 One-Hot Enconding

In [15]:
df['aux'] = df['tipo_residente'].apply(
    lambda column: [
        item.strip() for item in column
            .replace('(a)', '')
            .replace('-', '_')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('tipo_residente_')

insert_position = df.columns.get_loc('tipo_residente') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [16]:
df

,curso,serie,tipo_residente,tipo_residente_externo,tipo_residente_interno,tipo_residente_semi_residente,atv_culturais,hab_musicais,esportes,jogos,...,atv_culturais_a_fazer,hab_musicais_desejadas,esportes_desejados,jogos_desejados,lutas_desejadas,melhoria_estrutura,novas_estruturas,melhoria_alimentacao,transporte,projetos_a_participar
0,agro,1,interno,0,1,0,danca,bateria / percussao,"futebol de salao, basquete, handebol","jogos eletronicos, truco",...,teatro,bateria / percussao,atletismo,xadrez,muay thai,"recuperacao da sala de jogos, recuperacao do c...","areas para descanso para o intervalo, salas de...",lanches diarios,"criacao de uma ciclovia, para seguranca de que...",recuperacao e ocupacao do antigo cine januaria
1,agro,1,interno,0,1,0,pintura / desenho,"canto, violao / guitarra / demais instrumentos...","futebol de salao, handebol, peteca","jogos eletronicos, domino, truco",...,danca,piano / orgao / teclado,"voleibol, vaquejada / rodeio / cavalgada","xadrez, sinuca, pebolin (toto)",karate,"melhora na qualidade e alcance da internet, re...",areas para descanso para o intervalo,lanches diarios,nenhuma opcao,oficina de violao
2,agro,1,externo(a),1,0,0,"danca, teatro, artesanato",nenhuma opcao,peteca,"jogos eletronicos, uno",...,"danca, pintura / desenho, grafite / arte urban...",violao / guitarra / demais instrumentos de cor...,"voleibol, natacao","jogos eletronicos, truco, uno",boxe,"melhora na qualidade e alcance da internet, re...","areas para descanso para o intervalo, colchone...","opcoes vegetarianas no almoco, almoco para tod...",melhores condicoes dos onibus terceirizados (m...,recuperacao e ocupacao do antigo cine januaria
3,agro,1,externo(a),1,0,0,pintura / desenho,nenhuma opcao,"queimada, ciclismo",uno,...,"pintura / desenho, grafite / arte urbana, arte...",violao / guitarra / demais instrumentos de corda,voleibol,uno,nenhuma opcao,"melhora na qualidade e alcance da internet, re...","areas para descanso para o intervalo, salas de...","almoco para todos, lanches diarios, compra de ...","criacao de uma ciclovia, para seguranca de que...",recuperacao e ocupacao do antigo cine januaria
4,agro,1,externo(a),1,0,0,teatro,violao / guitarra / demais instrumentos de cor...,"futebol de campo, futebol de salao, voleibol, ...","jogos eletronicos, xadrez, dama, domino, truco...",...,"pintura / desenho, teatro",violao / guitarra / demais instrumentos de cor...,"basquete, futvolei, futmesa","jogos eletronicos, xadrez, dama, domino, truco...","karate, kung fu","melhora na qualidade e alcance da internet, su...",salas de aula ao ar livre,"opcoes vegetarianas no almoco, lanches diarios...","criacao de uma ciclovia, para seguranca de que...",oficina de violao
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,mamb,3,externo(a),1,0,0,danca,canto,"voleibol, handebol",nenhuma opcao,...,teatro,piano / orgao / teclado,nenhuma opcao,xadrez,muay thai,"melhora na qualidade e alcance da internet, ma...","areas para descanso para o intervalo, colchone...","almoco para todos, lanches diarios, compra de ...","criacao de uma ciclovia, para seguranca de que...",recuperacao e ocupacao do antigo cine januaria
607,mamb,3,externo(a),1,0,0,nenhuma opcao,nenhuma opcao,"voleibol, queimada","dama, domino, truco, uno, sinuca",...,nenhuma opcao,nenhuma opcao,nenhuma opcao,nenhuma opcao,nenhuma opcao,"melhora na qualidade e alcance da internet, ma...","areas para descanso para o intervalo, microond...","almoco para todos, lanches diarios, poder paga...","criacao de uma ciclovia, para seguranca de que...",nenhuma opcao
608,mamb,3,externo(a),1,0,0,poesia / literatura,nenhuma opcao,handebol,truco,...,nenhuma opcao,nenhuma opcao,nenhuma opcao,truco,nenhuma opcao,"melhora na qualidade e alcance da internet, ma...","areas para descanso para o intervalo, colchone...","almoco para todos, lanches diarios, compra de ...","criacao de uma ciclovia, para seguranca de que...",recuperacao e ocupacao do antigo cine januaria
609,mamb

##### atv_culturais

In [17]:
df['aux'] = df['atv_culturais'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('artes marciais', 'luta')
            .replace('capoeira', 'luta')
            .replace('karate', 'luta')
            .replace('canto', 'musica')
            .replace('outro: musica', 'musica')
            .replace('praticava futsal no ginasio da vila sao joao', 'futsal')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('atv_cultural_')

insert_position = df.columns.get_loc('atv_culturais') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [18]:
cols = df.columns[df.columns.str.startswith('atv_cultural_')].tolist()
cols

['atv_cultural_apresentacoes musicais',
 'atv_cultural_artesanato',
 'atv_cultural_crio historia e enigmas',
 'atv_cultural_danca',
 'atv_cultural_edicoes de video...',
 'atv_cultural_esporte',
 'atv_cultural_fotografia',
 'atv_cultural_futsal',
 'atv_cultural_grafite/arte urbana',
 'atv_cultural_luta',
 'atv_cultural_musica',
 'atv_cultural_natacao',
 'atv_cultural_nenhuma opcao',
 'atv_cultural_oficinas de gastronomia',
 'atv_cultural_pesquisa cientifica',
 'atv_cultural_piano',
 'atv_cultural_pintura de pano',
 'atv_cultural_pintura/desenho',
 'atv_cultural_poesia/literatura',
 'atv_cultural_producao de videos',
 'atv_cultural_teatro',
 'atv_cultural_violao']

##### hab_musicais

In [19]:
df['aux'] = df['hab_musicais'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('hab_musical_')

insert_position = df.columns.get_loc('hab_musicais') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [20]:
cols = df.columns[df.columns.str.startswith('hab_musical_')].tolist()
cols

['hab_musical_bateria/percussao',
 'hab_musical_berimbau',
 'hab_musical_canto',
 'hab_musical_carron',
 'hab_musical_instrumentos de sopro',
 'hab_musical_muito pouco.',
 'hab_musical_nenhuma opcao',
 'hab_musical_outro: meia lua',
 'hab_musical_piano/orgao/teclado',
 'hab_musical_repente',
 'hab_musical_rima',
 'hab_musical_sanfona',
 'hab_musical_sei muito pouco sobre violino',
 'hab_musical_violao/guitarra/demais instrumentos de corda']

##### esportes

In [21]:
df['aux'] = df['esportes'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('caminhada/corridaa', 'caminhada/corrida')
            .replace('futbol de campo', 'futebol de campo')
            .replace('futbol de salao', 'futebol de salao')
            .replace('handbol', 'handebol')
            .replace('musculacao', 'academia')
            .replace('karate', 'artes marciais')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('esporte_')

insert_position = df.columns.get_loc('esportes') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [22]:
cols = df.columns[df.columns.str.startswith('esporte_')].tolist()
cols

['esporte_academia',
 'esporte_artes marciais',
 'esporte_atletismo',
 'esporte_badminton',
 'esporte_basquete',
 'esporte_caminhada/corrida',
 'esporte_ciclismo',
 'esporte_futebol de campo',
 'esporte_futebol de salao',
 'esporte_futmesa',
 'esporte_futsal',
 'esporte_futvolei',
 'esporte_handebol',
 'esporte_natacao',
 'esporte_nenhuma opcao',
 'esporte_peteca',
 'esporte_pingue pongue',
 'esporte_queimada',
 'esporte_tenis',
 'esporte_vaquejada/rodeio/cavalgada',
 'esporte_voleibol']

##### jogos

In [23]:
df['aux'] = df['jogos'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('assim como o de mesa', 'rpg')
            .replace('filho da puta', 'nenhuma opcao')
            .replace('league of legends', 'jogos eletronicos')
            .replace('overwatch', 'jogos eletronicos')
            .replace("(toto)", '')
            .replace('pebolin', 'pebolin/toto')
            .replace('por exemplo o d&d', 'rpg')
            .replace('rpg variados', 'rpg')
            .replace('xadres', 'xadrez')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('jogo_')

insert_position = df.columns.get_loc('jogos') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [24]:
cols = df.columns[df.columns.str.startswith('jogo_')].tolist()
cols

['jogo_buraco',
 'jogo_dama',
 'jogo_domino',
 'jogo_jogos eletronicos',
 'jogo_nenhuma opcao',
 'jogo_pebolin/toto',
 'jogo_pif',
 'jogo_resta um',
 'jogo_rpg',
 'jogo_sinuca',
 'jogo_truco',
 'jogo_uno',
 'jogo_xadrez']

##### lutas

In [25]:
df['aux'] = df['lutas'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('ja pratiquei karate', 'karate')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('luta_')

insert_position = df.columns.get_loc('lutas') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [26]:
cols = df.columns[df.columns.str.startswith('luta_')].tolist()
cols

['luta_boxe',
 'luta_capoeira',
 'luta_jeet kune do',
 'luta_jiu-jitsu',
 'luta_judo',
 'luta_karate',
 'luta_krav maga',
 'luta_kung fu',
 'luta_muay thai',
 'luta_nenhuma opcao']

##### atv_culturais_a_fazer

In [27]:
df['aux'] = df['atv_culturais_a_fazer'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('aula de bateria', 'aprender algum instrumento')
            .replace('tocar teclado ou outros instrumentos.', 'aprender algum instrumento')
            .replace('(marquei poesia, mas sou boa em leitura, nao em escrever)', '')
            .replace('outras opcoes (especifique) ou observacoes', '')
            .replace('karate', 'luta')
            .replace('jiu-jitsu', 'luta')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('atv_culturais_a_fazer_')

insert_position = df.columns.get_loc('atv_culturais_a_fazer') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [28]:
cols = df.columns[df.columns.str.startswith('atv_culturais_a_fazer_')].tolist()
cols

['atv_culturais_a_fazer_aprender algum instrumento',
 'atv_culturais_a_fazer_apresentacoes musicais',
 'atv_culturais_a_fazer_artesanato',
 'atv_culturais_a_fazer_danca',
 'atv_culturais_a_fazer_fotografia',
 'atv_culturais_a_fazer_grafite/arte urbana',
 'atv_culturais_a_fazer_luta',
 'atv_culturais_a_fazer_musica',
 'atv_culturais_a_fazer_nenhuma opcao',
 'atv_culturais_a_fazer_oficina de gastronomia',
 'atv_culturais_a_fazer_pintura/desenho',
 'atv_culturais_a_fazer_poesia/literatura',
 'atv_culturais_a_fazer_teatro']

##### hab_musicais_desejadas

In [29]:
df['aux'] = df['hab_musicais_desejadas'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('percussao', '')
            .replace('percussa', '')
            .replace('bateria/', 'bateria/percussao')
            .replace('violao/guitarra/demais instrumentos de corda', 'canto, instrumentos de corda')
            .replace('canto/violao/guitarra/demais instrumentos de corda', 'canto/canto, instrumentos de corda')
            .replace('canto/canto', 'canto')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('hab_musicais_desejadas_')

insert_position = df.columns.get_loc('hab_musicais_desejadas') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [30]:
cols = df.columns[df.columns.str.startswith('hab_musicais_desejadas_')].tolist()
cols

['hab_musicais_desejadas_artes digitais',
 'hab_musicais_desejadas_bateria/percussao',
 'hab_musicais_desejadas_canto',
 'hab_musicais_desejadas_desenho',
 'hab_musicais_desejadas_instrumentos de corda',
 'hab_musicais_desejadas_instrumentos de sopro',
 'hab_musicais_desejadas_kalimba',
 'hab_musicais_desejadas_nenhuma opcao',
 'hab_musicais_desejadas_pandeiro',
 'hab_musicais_desejadas_piano/orgao/teclado',
 'hab_musicais_desejadas_rima',
 'hab_musicais_desejadas_sanfona']

##### esportes_desejados

In [31]:
df['aux'] = df['esportes_desejados'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('futbol de campo', 'futebol de campo')
            .replace('futebol de campo/basquete/handebol/pingue pongue/peteca/futmesa', 'futebol de campo, basquete, handebol, pingue pongue, peteca, futmesa')
            .replace('futvilei', 'futvolei')
            .replace('handbol', 'handebol')
            .replace('queimada e atletismo', 'queimada, atletismo')
            .replace('voleiboll', 'voleibol')
            .replace('voleiwheeling e moto crossboll', 'wheeling/moto cross')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('esportes_desejados_')

insert_position = df.columns.get_loc('esportes_desejados') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [32]:
cols = df.columns[df.columns.str.startswith('esportes_desejados_')].tolist()
cols

['esportes_desejados_atletismo',
 'esportes_desejados_badminton',
 'esportes_desejados_basquete',
 'esportes_desejados_beatch tenis',
 'esportes_desejados_caminhada/corrida',
 'esportes_desejados_ciclismo',
 'esportes_desejados_futebol de campo',
 'esportes_desejados_futebol de salao',
 'esportes_desejados_futmesa',
 'esportes_desejados_futvolei',
 'esportes_desejados_handebol',
 'esportes_desejados_natacao',
 'esportes_desejados_nenhuma opcao',
 'esportes_desejados_peteca',
 'esportes_desejados_pingue pongue',
 'esportes_desejados_queimada',
 'esportes_desejados_tenis',
 'esportes_desejados_vaquejada/rodeio/cavalgada',
 'esportes_desejados_voleibol',
 'esportes_desejados_wheeling e moto cross']

##### jogos_desejados

In [33]:
df['aux'] = df['jogos_desejados'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace("(toto)", '')
            .replace('pebolin', 'pebolin/toto')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('jogos_desejados_')

insert_position = df.columns.get_loc('jogos_desejados') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [34]:
cols = df.columns[df.columns.str.startswith('jogos_desejados_')].tolist()
cols

['jogos_desejados_dama',
 'jogos_desejados_domino',
 'jogos_desejados_jogos eletronicos',
 'jogos_desejados_nenhuma opcao',
 'jogos_desejados_pebolin/toto',
 'jogos_desejados_pif',
 'jogos_desejados_sinuca',
 'jogos_desejados_truco',
 'jogos_desejados_uno',
 'jogos_desejados_xadrez']

##### lutas_desejadas

In [35]:
df['aux'] = df['lutas_desejadas'].apply(
    lambda column: [
        item.strip() for item in column
            .replace(' / ', '/')
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace("kung fuh", 'kung fu')
            .split(',')
    ]
)

dummies = df['aux'].str.join('|').str.get_dummies(sep='|').add_prefix('lutas_desejadas_')

insert_position = df.columns.get_loc('lutas_desejadas') + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)
df.drop('aux', axis=1, inplace=True)

In [36]:
cols = df.columns[df.columns.str.startswith('lutas_desejadas_')].tolist()
cols

['lutas_desejadas_boxe',
 'lutas_desejadas_capoeira',
 'lutas_desejadas_jiu-jitsu',
 'lutas_desejadas_judo',
 'lutas_desejadas_karate',
 'lutas_desejadas_kung fu',
 'lutas_desejadas_muay thai',
 'lutas_desejadas_nenhuma opcao',
 'lutas_desejadas_taekwondo']

##### melhoria_estrutura

In [37]:
df[df['melhoria_estrutura'].astype(str).str.contains('substituicao dos ventiladores por ar condicionado ou novos ventiladores\nrecuperacao da sala de jogos\nrecuperacao das quadras')]

,curso,serie,tipo_residente,tipo_residente_externo,tipo_residente_interno,tipo_residente_semi_residente,atv_culturais,atv_cultural_apresentacoes musicais,atv_cultural_artesanato,atv_cultural_crio historia e enigmas,...,lutas_desejadas_karate,lutas_desejadas_kung fu,lutas_desejadas_muay thai,lutas_desejadas_nenhuma opcao,lutas_desejadas_taekwondo,melhoria_estrutura,novas_estruturas,melhoria_alimentacao,transporte,projetos_a_participar
440,mamb,1,interno,0,1,0,teatro,0,0,0,...,0,0,0,0,0,"melhora na qualidade e alcance da internet,ref...","areas para descanso para o intervalo, colchone...","opcoes vegetarianas no almoco, compra de tickt...","criacao de uma ciclovia, para seguranca de que...",nenhuma opcao


In [38]:
df.loc[440, 'melhoria_estrutura'] = df.loc[440, 'melhoria_estrutura'].replace('\n', ', ')

In [39]:
df["aux"] = df["melhoria_estrutura"].apply(
    lambda column: [
        item.strip() for item in column
            .replace(" / ", "/")
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('acesso a academia de ginastica durante o dia intervalo) e fins-de-semana', 'acesso a academia de ginastica durante o dia e fins-de-semana')
            .replace('melhoria do bebedouro para agua gelada', 'melhoria do bebedouro')
            .replace('agua gelada', 'melhoria do bebedouro')
            .replace('melhora nos banheiros em geral', 'melhoria nos banheiros')
            .replace('banheiro dos semi-ressidente', 'melhoria nos banheiros')
            .replace('reforma dos banheiros do ginasio', 'melhoria nos banheiros')
            .replace('termos lanches na parte da tarde', 'ter lanche no horario da tarde e cafe da manha')
            .replace(
                'ter lanche no horario da tarde e cafe da manha (pois tem gente que passa mal sem comer o dia todo e nao e todo mundo que tem dinheiro para comprar lanche ou algo do tipo',
                'ter lanche no horario da tarde e cafe da manha',
            )
            .split(",")
    ]
)

dummies = (df["aux"].str.join("|").str.get_dummies(sep="|").add_prefix("melhoria_estrutura_"))

insert_position = df.columns.get_loc("melhoria_estrutura") + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)

df.drop(columns={"aux", "melhoria_estrutura_pois na tarde nos passamos fome."}, axis=1, inplace=True)

In [40]:
cols = df.columns[df.columns.str.startswith('melhoria_estrutura_')].tolist()
cols

['melhoria_estrutura_acesso a academia de ginastica durante o dia e fins-de-semana',
 'melhoria_estrutura_colocar mais bancos e substituicao dos bancos que estao quebrados.',
 'melhoria_estrutura_construcao de piscinas para a pratica de natacao.',
 'melhoria_estrutura_internet nos quarto',
 'melhoria_estrutura_mais instrumentos para a fanfarra',
 'melhoria_estrutura_melhora na qualidade e alcance da internet',
 'melhoria_estrutura_melhoria do bebedouro',
 'melhoria_estrutura_melhoria nos banheiros',
 'melhoria_estrutura_nenhuma opcao',
 'melhoria_estrutura_plantacao de mais arvores',
 'melhoria_estrutura_recuperacao da area de atletismo',
 'melhoria_estrutura_recuperacao da sala de jogos',
 'melhoria_estrutura_recuperacao da sala de musica',
 'melhoria_estrutura_recuperacao das quadras',
 'melhoria_estrutura_recuperacao do campo de futebol',
 'melhoria_estrutura_substituicao dos ventiladores por ar condicionado ou novos ventiladores',
 'melhoria_estrutura_ter acesso livre aos chuveiros

##### novas_estruturas

In [41]:
df["aux"] = df["novas_estruturas"].apply(
    lambda column: [
        item.strip() for item in column
            .replace(" / ", "/")
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('cadeiras e mesas soltas (as duas juntas nao esta dando certo)', 'cadeiras e mesas soltas')
            .replace(
                'armarios para colocar as mochilas por que levar peso e ter suas costas doendo e uma coisa muito complicada (ate por que ninguem vai querer fornecer dinheiro para pagar medico',
                'armarios para colocar as mochilas'
            )
            .split(",")
    ]
)

dummies = (df["aux"].str.join("|").str.get_dummies(sep="|").add_prefix("novas_estruturas_"))

insert_position = df.columns.get_loc("novas_estruturas") + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)

df.drop(columns={
    'aux',
    'novas_estruturas_como embaixo das arvores',
    'novas_estruturas_e complicado andar com a mochila lotada de materias e livros de um lado para o outro.',
    'novas_estruturas_ja que passamos o dia inteiro la',
    'novas_estruturas_o professor alberto berto da agroecologia estava com pensamentos que possibilitasse seu setor receber alunos com uma maior comodidade para descanso .',
    'novas_estruturas_principalmente para as turmas que tem aulas praticas e aula de educacao fisica no mesmo dia.',
    'novas_estruturas_remedio ou fisioterapeuta para tal estudante)...',
}, axis=1, inplace=True)

In [42]:
cols = df.columns[df.columns.str.startswith('novas_estruturas_')].tolist()
cols

['novas_estruturas_ar-condicionado nas salas de aula',
 'novas_estruturas_areas de lazer em lugares frescos',
 'novas_estruturas_areas para descanso para o intervalo',
 'novas_estruturas_armarios para armazenar os nossos materiais',
 'novas_estruturas_armarios para colocar as mochilas',
 'novas_estruturas_armarios para os alunos guardarem roupas no dia de educacao fisica',
 'novas_estruturas_cadeiras e mesas soltas',
 'novas_estruturas_colchonetes ou redes',
 'novas_estruturas_microondas para esquentar as marmitas',
 'novas_estruturas_nenhuma opcao',
 'novas_estruturas_sala de jogos com diversas modalidades.',
 'novas_estruturas_salas de aula ao ar livre']

##### melhoria_alimentacao

In [43]:
df['melhoria_alimentacao'] = df['melhoria_alimentacao'].str.replace('...', ', ', regex=False)

In [44]:
df["aux"] = df["melhoria_alimentacao"].apply(
    lambda column: [
        item.strip() for item in column
            .replace(" / ", "/")
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('(se nao possivel os lanches diarios) ter mais uma lanchonete', 'ter mais uma lanchonete')
            .replace('compra de ticktes on-line (sem precisar enfrentar fila)', 'compra de ticktes on-line')
            .replace('melhorias nos sucos do almoco ou lanche', 'melhoria no suco')
            .replace('suco bom', 'melhoria no suco')
            .replace('outro:self-service', 'self-service')
            .replace('volta da opcao em que podiamos colocar o quanto quisermos no almoco', 'self-service')
            .replace('volta do self service', 'self-service')
            .replace('voltar a gente mesmo se servir', 'self-service')
            .replace('que pudessem avisar por exemplo quando tem abacaxi na salada pois muitos alunos sao alergicos', 'aviso sobre alérgenos nas refeições')
            .replace('poder pagar com pix e/ou cartao (almoco e demais produtos do ponto de venda)', 'poder pagar com pix e/ou cartao')
            .replace('poder pagar no pix e/ou cartao', 'poder pagar com pix e/ou cartao')
            .replace(
                'poder pagar com pix e extremamente importante pois estou querendo comprar alguns produtos do posto de vendas mas so tenho pix no momento',
                'poder pagar com pix e/ou cartao'
            )
            .split(",")
    ]
)

dummies = (df["aux"].str.join("|").str.get_dummies(sep="|").add_prefix("melhoria_alimentacao_"))

insert_position = df.columns.get_loc("melhoria_alimentacao") + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)

df.drop(columns={
    "aux",
    'melhoria_alimentacao_em virtude de ser servido somente uma refeicao para os alunos que por sua vez ficam o dia inteiro dentro da instituicao.',
    'melhoria_alimentacao_o suco daqui e horrivel.',
    'melhoria_alimentacao_pois as vezes os sucos que fazem tem um gosto nada agradavel',
    'melhoria_alimentacao_para nao chegar atrasado na aula por causa de fila.',
    'melhoria_alimentacao_pois eu como pouco e sempre colocam muito e desperdisa',
}, axis=1, inplace=True)

In [45]:
cols = df.columns[df.columns.str.startswith('melhoria_alimentacao_')].tolist()
cols

['melhoria_alimentacao_almoco para todos',
 'melhoria_alimentacao_aumento na quantidade de tickets para venda',
 'melhoria_alimentacao_aviso sobre alérgenos nas refeições',
 'melhoria_alimentacao_compra de ticktes on-line',
 'melhoria_alimentacao_deveria ter almoco gratis para todos',
 'melhoria_alimentacao_lanches diarios',
 'melhoria_alimentacao_mais opcoes de saladas',
 'melhoria_alimentacao_melhoria no suco',
 'melhoria_alimentacao_nenhuma opcao',
 'melhoria_alimentacao_opcoes a carne de porco no almoco (adventistas)',
 'melhoria_alimentacao_opcoes vegetarianas no almoco',
 'melhoria_alimentacao_poder pagar com pix e/ou cartao',
 'melhoria_alimentacao_poder repetir a comida',
 'melhoria_alimentacao_self-service',
 'melhoria_alimentacao_ter mais uma lanchonete']

##### transporte

In [46]:
df["aux"] = df["transporte"].apply(
    lambda column: [
        item.strip() for item in column
            .replace(" / ", "/")
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('melhores condicoes dos onibus terceirizados (mais lugares', 'melhores condicoes dos onibus terceirizados')
            .replace('menor preco na passagem', 'menor preco')
            .replace('menor preco', 'melhoria no preco da passagem')
            .split(",")
    ]
)

dummies = (df["aux"].str.join("|").str.get_dummies(sep="|").add_prefix("transporte_"))

insert_position = df.columns.get_loc("transporte") + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)

df.drop(columns={
    "aux",
    'transporte_estao poucos para a quantidade de alunos e ficam superlotados',
    'transporte_onibus melhores e mais limpos)',
    'transporte_para evitar sair correndo no ultimo horario',
    'transporte_para seguranca de quem vem de bicicleta',
}, axis=1, inplace=True)

In [47]:
cols = df.columns[df.columns.str.startswith('transporte_')].tolist()
cols

['transporte_criacao de uma ciclovia',
 'transporte_horario fixo de saida dos onibus',
 'transporte_mais horarios de onibus',
 'transporte_mais onibus',
 'transporte_mais um onibus para a rota do alto.',
 'transporte_melhores condicoes dos onibus terceirizados',
 'transporte_melhoria no preco da passagem',
 'transporte_nenhuma opcao']

##### transporte

In [48]:
df["aux"] = df["projetos_a_participar"].apply(
    lambda column: [
        item.strip() for item in column
            .replace(" / ", "/")
            .replace('/ ', '/')
            .replace(' /', '/')
            .replace('ocupacao e recuperacao do antigo cine januaria', 'recuperacao e ocupacao do antigo cine januaria')
            .replace('radio intervalo (um canal de radio exclusivo do campus)', 'radio do campus')
            .replace('seria legal adicionar secoes de rpg de mesa para quem quisesse participar.', 'adicionar sessões de RPG.')
            .split(",")
    ]
)

dummies = (df["aux"].str.join("|").str.get_dummies(sep="|").add_prefix("projetos_a_participar_"))

insert_position = df.columns.get_loc("projetos_a_participar") + 1

df = pd.concat([df.iloc[:, :insert_position], dummies, df.iloc[:, insert_position:]], axis=1)

df.drop(columns={
    "aux",
    'projetos_a_participar_outro:',
}, axis=1, inplace=True)

In [49]:
cols = df.columns[df.columns.str.startswith('projetos_a_participar_')].tolist()
cols

['projetos_a_participar_adicionar sessões de RPG.',
 'projetos_a_participar_aula de volei para todos',
 'projetos_a_participar_nenhuma opcao',
 'projetos_a_participar_oficina de teclado',
 'projetos_a_participar_oficina de violao',
 'projetos_a_participar_radio do campus',
 'projetos_a_participar_recuperacao e ocupacao do antigo cine januaria',
 'projetos_a_participar_time de e-sports do campus e torneios intercampis.']

In [50]:
# Analisar viabilidade de usar essa técnica aqui também
# def melt_columns(prefix: str, var_name: str, value_name: str):
#     return df.melt(
#         id_vars=["curso"],
#         value_vars=[col for col in df.columns if col.startswith(prefix)],
#         var_name=var_name,
#         value_name=value_name,
#     )

# atv_melt = melt_columns("atv_cultural_", "atividade", "tipo")


#### 🗃️ Bases - Criar bases para aplicação dos algoritmos

In [51]:
df.drop(columns=[
    'tipo_residente',
    'atv_culturais',
    'hab_musicais',
    'esportes',
    'jogos',
    'lutas',
    'atv_culturais_a_fazer',
    'hab_musicais_desejadas',
    'esportes_desejados',
    'jogos_desejados',
    'lutas_desejadas',
    'melhoria_estrutura',
    'novas_estruturas',
    'melhoria_alimentacao',
    'transporte',
    'projetos_a_participar'
], inplace=True)

df.to_csv(f'{GENERATED_DATASET_PATH}/df.csv', index=False)

In [52]:
# Buscar os índices de cada coluna para facilitar a divisão das bases de dados
df_cols = pd.DataFrame([[column] for column in df.columns], columns=['columns'])
df_cols

,columns
0,curso
1,serie
2,tipo_residente_externo
3,tipo_residente_interno
4,tipo_residente_semi_residente
...,...
206,projetos_a_participar_oficina de teclado
207,projetos_a_participar_oficina de violao
208,projetos_a_participar_radio do campus
209,projetos_a_participar_recuperacao e ocupacao d...


##### base das perguntas sobre as atividades que o aluno faz - colunas 0-84

In [53]:
df_atv = df.iloc[:, :85].copy()
df_atv.to_csv(f'{GENERATED_DATASET_PATH}/df_atv.csv', index=False)
df_atv

,curso,serie,tipo_residente_externo,tipo_residente_interno,tipo_residente_semi_residente,atv_cultural_apresentacoes musicais,atv_cultural_artesanato,atv_cultural_crio historia e enigmas,atv_cultural_danca,atv_cultural_edicoes de video...,...,luta_boxe,luta_capoeira,luta_jeet kune do,luta_jiu-jitsu,luta_judo,luta_karate,luta_krav maga,luta_kung fu,luta_muay thai,luta_nenhuma opcao
0,agro,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,agro,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,agro,1,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,agro,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,agro,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,mamb,3,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
607,mamb,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
608,mamb,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
609,mamb,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


##### base das perguntas sobre as atividades culturais que o aluno deseja aprender - colunas 0-4, 85-148

In [54]:
df_atv_a_fazer = pd.concat([df.iloc[:, :5], df.iloc[:, 85:149]], axis=1)
df_atv_a_fazer.to_csv(f'{GENERATED_DATASET_PATH}/df_atv_a_fazer.csv', index=False)
df_atv_a_fazer

,curso,serie,tipo_residente_externo,tipo_residente_interno,tipo_residente_semi_residente,atv_culturais_a_fazer_aprender algum instrumento,atv_culturais_a_fazer_apresentacoes musicais,atv_culturais_a_fazer_artesanato,atv_culturais_a_fazer_danca,atv_culturais_a_fazer_fotografia,...,jogos_desejados_xadrez,lutas_desejadas_boxe,lutas_desejadas_capoeira,lutas_desejadas_jiu-jitsu,lutas_desejadas_judo,lutas_desejadas_karate,lutas_desejadas_kung fu,lutas_desejadas_muay thai,lutas_desejadas_nenhuma opcao,lutas_desejadas_taekwondo
0,agro,1,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,agro,1,0,1,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
2,agro,1,1,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,agro,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,agro,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,mamb,3,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
607,mamb,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
608,mamb,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
609,mamb,3,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0


##### base das perguntas sobre estrutura - colunas 0-4, 149:

In [55]:
df_estrutura = pd.concat([df.iloc[:, :5], df.iloc[:, 149:]], axis=1)
df_estrutura.to_csv(f'{GENERATED_DATASET_PATH}/df_estrutura.csv', index=False)
df_estrutura

,curso,serie,tipo_residente_externo,tipo_residente_interno,tipo_residente_semi_residente,melhoria_estrutura_acesso a academia de ginastica durante o dia e fins-de-semana,melhoria_estrutura_colocar mais bancos e substituicao dos bancos que estao quebrados.,melhoria_estrutura_construcao de piscinas para a pratica de natacao.,melhoria_estrutura_internet nos quarto,melhoria_estrutura_mais instrumentos para a fanfarra,...,transporte_melhoria no preco da passagem,transporte_nenhuma opcao,projetos_a_participar_adicionar sessões de RPG.,projetos_a_participar_aula de volei para todos,projetos_a_participar_nenhuma opcao,projetos_a_participar_oficina de teclado,projetos_a_participar_oficina de violao,projetos_a_participar_radio do campus,projetos_a_participar_recuperacao e ocupacao do antigo cine januaria,projetos_a_participar_time de e-sports do campus e torneios intercampis.
0,agro,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,agro,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,agro,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,agro,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,agro,1,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,mamb,3,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
607,mamb,3,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
608,mamb,3,1,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
609,mamb,3,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


##### base para dashboard

In [56]:
# df.drop(columns=[
#     # 'atv_culturais',
#     # 'hab_musicais',
#     # 'esportes',
#     # 'jogos',
#     # 'lutas',
#     # 'atv_culturais_a_fazer',
#     # 'hab_musicais_desejadas',
#     # 'esportes_desejados',
#     # 'jogos_desejados',
#     # 'lutas_desejadas',
#     # Analisar como fazer a tratativa nessas colunas ao invés de remover
#     # 'melhoria_estrutura',
#     # 'novas_estruturas',
#     # # 'melhoria_alimentacao',
#     # 'transporte',
#     # 'projetos_a_participar',
# ], inplace=True)

df.to_csv(f'{GENERATED_DATASET_PATH}/data.csv', index=False)